In [ ]:
import pandas as pd
import os, json, re, math, time
from textwrap import dedent
from collections import Counter
from typing import List, Dict, Tuple, Optional, Any
from dotenv import load_dotenv
from openai import OpenAI
from huggingface_hub import InferenceClient

import openai
import base64
import requests
from pathlib import Path

In [ ]:
load_dotenv()  # loads OPENAI_API_KEY from .env if present

# Create a single client
# client = OpenAI()

print("Has OPENAI_API_KEY:", bool(os.getenv("OPENAI_API_KEY")))
print("Has huggingface token:", bool(os.getenv("HF_TOKEN")))

HF_TOKEN = os.getenv("HF_TOKEN")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

sd_client = InferenceClient(
    provider="nscale",
    api_key=HF_TOKEN,
)

Has OPENAI_API_KEY: True
Has huggingface token: True


In [ ]:
# ================================
# Prompt Dictionary for All Products
# ================================
PRODUCT_PROMPTS = {

    # -------------------------
    # Product 1: Shiatsu Massager
    # -------------------------
    "product1_massager": {
        # "v1": """Create an image of a compact, pillow-like back and neck massager in rich brown fabric.
        # The massager should feature soft silicone nodes on each side, an ergonomic design that fits body
        # contours, and Velcro straps for securing it to a chair. Include user-friendly buttons and a
        # zipper for a replacement cover. The overall look should convey a sturdy and modern aesthetic,
        # suitable for therapeutic use in home or office settings.""",

#         "v2": """Ultra-realistic photo of an ergonomic shiatsu massage pillow.
# Brown faux leather + fabric material, curved compact shape, four rounded massage nodes,
# simple control buttons, velcro straps. Bright studio lighting, sharp detail.""",

        "v3": """Realistic studio product photo of a compact, pillow-shaped shiatsu back and neck massager.
Key visual features:
- rich brown fabric cover made of nylon/polyester
- four raised soft silicone massage nodes forming smooth rounded bumps
- ergonomic curved design that fits body contours
- Velcro straps on the back for attaching to a chair
- user-friendly side control buttons and a visible zipper for a replaceable cover
Style: clean white seamless background, soft diffused lighting, crisp detail and accurate textures."""
    },

    # -------------------------
    # Product 2: Retro Mechanical Keyboard
    # -------------------------
    "product2_keyboard": {
#         "v1": """Create a image of a retro-style audio device with a boxy shape made of thick plastic.
#         The device has a creamy grey body adorned with bold red accents. It features large 'Super Buttons' on the front,
#         a prominent central volume knob, and a soft glowing power LED indicator. The overall design
#         exudes a nostalgic aesthetic reminiscent of classic electronics, highlighting concave keys and a user-friendly layout.""",

#         "v2": """Ultra-realistic retro-style mechanical keyboard modeled after classic 8-bit consoles.
# Matte creamy grey ABS plastic housing, bold red accent buttons, oversized Super Buttons,
# concave retro keycaps, a metallic central volume knob, and an illuminated LED indicator.
# Bright studio lighting, sharp detail.""",

        "v3": """Realistic studio product photo of a retro mechanical keyboard inspired by classic 1980s gaming consoles.
Key visual features:
- thick boxy plastic chassis in creamy grey with bold red accents
- two oversized circular “Super Buttons” on the front panel
- concave mechanical keycaps arranged in a compact layout
- a prominent central round volume knob
- a small softly glowing LED power indicator
Style: white seamless background, bright studio lighting, sharp edges and clear plastic texture detail."""
    },

    # -------------------------
    # Product 3: Hario V60 Starter Kit
    # -------------------------
    "product3_hario": {
        # "v1": """Create a image of a clean, high-resolution product photo of a Hario-style V60 pour-over
        # starter kit on a white seamless background. The kit includes a thick, sturdy white ceramic cone
        # dripper with V60 spiral ribs, a clear glass server marked up to about 600 ml (18 fl oz), a bag
        # of white paper filters (100-pack), and a simple coffee scoop. Emphasize the smooth ceramic texture,
        # the transparency and reflections on the glass server, and the minimal, modern Japanese aesthetic.
        # Soft diffused lighting, sharp focus, no clutter.""",

#         "v2": """Ultra-realistic studio photo of a manual pour-over coffee set.
# Thick white ceramic V60 dripper, transparent glass server with handle and printed markings,
# white cone-shaped filters, and a plastic scoop. Clean bright lighting, crisp detail.""",

        "v3": """Realistic high-resolution studio photo of a V60-style pour-over coffee starter kit.
Key components:
- a thick white ceramic V60 dripper with a conical form and spiral internal ribs
- a clear glass server with a curved handle and measurement markings up to ~600 ml
- a stack or bag of white V60 paper filters (100-pack)
- a simple plastic coffee scoop
Style: minimal modern Japanese aesthetic, white seamless background, soft diffused lighting, clean reflections and crisp detail."""
    }
}

In [ ]:
# OpenAI Image Generator
def generate_openai(prompt, output_path):
    print(f"[OpenAI] Generating {output_path} ...")
    response = openai.images.generate(
        model="gpt-image-1",
        prompt=prompt,
        size="1024x1024",
        n=1
    )
    img_b64 = response.data[0].b64_json
    Path(output_path).write_bytes(base64.b64decode(img_b64))
    print(f"[OpenAI] Saved -> {output_path}")

In [ ]:
# Stable Diffusion (SDXL) Generator
def generate_sdxl(prompt, output_path):
    print(f"[SDXL] Generating {output_path} ...")
    image = sd_client.text_to_image(
        prompt,
        model="stabilityai/stable-diffusion-xl-base-1.0"
    )
    image.save(output_path)
    print(f"[SDXL] Saved -> {output_path}")

In [ ]:
for product_name, versions in PRODUCT_PROMPTS.items():
    for version_tag, prompt in versions.items():

        # Output file names
        openai_output = f"images/{product_name}_openai_{version_tag}.png"
        sdxl_output = f"images/{product_name}_sdxl_{version_tag}.png"

        # Generate images
        generate_openai(prompt, openai_output)
        generate_sdxl(prompt, sdxl_output)

[OpenAI] Generating images/product1_massager_openai_v3.png ...
[OpenAI] Saved -> images/product1_massager_openai_v3.png
[SDXL] Generating images/product1_massager_sdxl_v3.png ...
[SDXL] Saved -> images/product1_massager_sdxl_v3.png
[OpenAI] Generating images/product2_keyboard_openai_v3.png ...
[OpenAI] Saved -> images/product2_keyboard_openai_v3.png
[SDXL] Generating images/product2_keyboard_sdxl_v3.png ...
[SDXL] Saved -> images/product2_keyboard_sdxl_v3.png
[OpenAI] Generating images/product3_hario_openai_v3.png ...
[OpenAI] Saved -> images/product3_hario_openai_v3.png
[SDXL] Generating images/product3_hario_sdxl_v3.png ...
[SDXL] Saved -> images/product3_hario_sdxl_v3.png
